### 데이터 받아오기

In [51]:
import json
import numpy as np
import random
import nltk

In [52]:
with open("Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [53]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

### 데이터 분리하기

In [54]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [55]:
reviewText = []
for i in range(len(review_data)):
    reviewText.append(review_data[i]["reviewText"])

In [56]:
y_data = []
for i in range(len(review_data)):
    y_data.append(review_data[i]["overall"])

### 가장 작은 평점으로 갯수 맞추기

In [57]:
from collections import Counter
Counter(y_data)

Counter({1.0: 323833, 2.0: 415110, 3.0: 955189, 4.0: 2223094, 5.0: 4980815})

In [58]:
from collections import Counter
length=sorted(Counter(y_data).most_common())[0][1]

In [59]:
search=[1.0,2.0,3.0,4.0,5.0]
count=0
text=[]
y=[]

for i in search:
    count=0
    for index,value in enumerate(y_data):
        if (count==length): break
        elif (i==value):
            text.append(reviewText[index])
            y.append(value)
            count=count+1

In [60]:
Counter(y)

Counter({1.0: 323833, 2.0: 323833, 3.0: 323833, 4.0: 323833, 5.0: 323833})

### 데이터 셔플

In [61]:
c = list(zip(text, y))
random.shuffle(c)

In [62]:
a, b = zip(*c)

In [63]:
text=list(a)
overall=np.array(list(b))

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
#vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(text)

In [65]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall , test_size=0.33, random_state=100)

In [66]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1084840, 675048), (1084840,), (534325, 675048), (534325,))

### 예측

#### Multiclass SVM

In [67]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

result = OneVsRestClassifier(LinearSVC(random_state=100)).fit(x_train, y_train)
result.predict(x_test)

array([ 4.,  2.,  1., ...,  3.,  1.,  3.])

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(result.predict(x_test),y_test)

0.55394750386001035

### LogisticRegression

In [69]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(x_train, y_train)
logreg.predict(x_test)==y_test

array([ True,  True,  True, ..., False,  True, False], dtype=bool)

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

0.56850605904646045

In [71]:
print(y_test[:10], logreg.predict(x_test)[:10])

[ 4.  2.  1.  5.  5.  5.  1.  3.  1.  5.] [ 4.  2.  1.  4.  5.  5.  1.  1.  3.  4.]


In [72]:
logreg.predict(x_test)[:10]

array([ 4.,  2.,  1.,  4.,  5.,  5.,  1.,  1.,  3.,  4.])

### INPUT 

In [73]:
texts = ["I hate this story. It's too boring. So I don't recommend this book to you guys",
        "I love this book"]
vecs = vectorizer.transform(texts) 

In [74]:
print(logreg.predict(vecs))

[ 1.  5.]
